In [21]:
import sqlite3
import pandas as pd

# Подключение к базе данных Sqlite
conn = sqlite3.connect('./data/finance.lite')

# Чтение таблицы 'transactions' в DataFrame
v_df = pd.read_sql_query("SELECT * FROM trans_all", conn)

# Закрытие соединения с базой данных
conn.close()

v_df.date = pd.to_datetime(v_df.date, format='%Y-%m-%d %H:%M:%S')

# Вывод первых нескольких строк DataFrame для проверки
# v_df
# v_df.info()
v_df.loc[v_df.payment == 0, 'category'].unique()




array(['Зарплата Д', '#Н/Д', 'Детские', 'Домашнее', 'Развлечения',
       'Межсчета', 'Проценты', 'Зарплата', 'Корректировка', 'Подарки',
       'Долги', 'Кэшбэк', 'Дивиденды', 'Продажа', 'Кредит', 'Заначка',
       'Обналичка'], dtype=object)

In [34]:
v_df_month_in = (v_df
                .loc[(v_df['payment'] == 1)]
                # .loc[(v_df['payment'] == 0) & ((v_df['category'] == 'Зарплата Д') | (v_df['category'] == 'Зарплата'))]
                # .groupby(['category', pd.Grouper(key='date', freq='M')])
              .groupby(['category', v_df['date'].dt.to_period('M')])
              ['amount']
              .sum()
              .reset_index()
              )
# print(v_df_month_in.values)
v_df_month_in.values

array([['#Н/Д', Period('2023-12', 'M'), 235000.0],
       ['0', Period('2024-12', 'M'), 12099.0],
       ['Авто', Period('2020-01', 'M'), 408155.52],
       ...,
       ['Электроника', Period('2023-06', 'M'), 12097.0],
       ['Электроника', Period('2023-07', 'M'), 35715.0],
       ['Электроника', Period('2023-08', 'M'), 73574.44]],
      shape=(1125, 3), dtype=object)